# Deep Learning for Natural Language Processing with Pytorch
This tutorial will walk you through the key ideas of deep learning programming using Pytorch. Many of the concepts (such as the computation graph abstraction and autograd) are not unique to Pytorch and are relevant to any deep learning tool kit out there

In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## 1. Torch tensor

In [4]:
# Create a torch.Tensor object with the given data.  It is a 1D vector
V_data = [1., 2., 3.]
V = torch.Tensor(V_data)
print(V)

# Creates a matrix
M_data = [[1., 2., 3.], [4., 5., 6]]
M = torch.Tensor(M_data)
print (M)

# Create a 3D tensor of size 2x2x2.
T_data = [[[1.,2.], [3.,4.]],
          [[5.,6.], [7.,8.]]]
T = torch.Tensor(T_data)
print(T)


 1
 2
 3
[torch.FloatTensor of size 3]


 1  2  3
 4  5  6
[torch.FloatTensor of size 2x3]


(0 ,.,.) = 
  1  2
  3  4

(1 ,.,.) = 
  5  6
  7  8
[torch.FloatTensor of size 2x2x2]



## 2. Computation Graph and automatic differentiation

In [8]:
# Variables wrap tensor objects
x = autograd.Variable( torch.Tensor([1., 2., 3]), requires_grad=True )
# You can access the data with the .data attribute
print (x.data)

# You can also do all the same operations you did with tensors with Variables.
y = autograd.Variable( torch.Tensor([4., 5., 6]), requires_grad=True )
z = x + y
print (z.data)

# BUT z knows something extra.
print (z.grad_fn)


 1
 2
 3
[torch.FloatTensor of size 3]


 5
 7
 9
[torch.FloatTensor of size 3]



## Logistic Regression Bag-of-words classifier

In [10]:
data = [ ("me gusta comer en la cafeteria".split(), "SPANISH"),
         ("Give it to me".split(), "ENGLISH"),
         ("No creo que sea una buena idea".split(), "SPANISH"),
         ("No it is not a good idea to get lost at sea".split(), "ENGLISH") ]

test_data = [ ("Yo creo que si".split(), "SPANISH"),
              ("it is lost on me".split(), "ENGLISH")]

word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

{'cafeteria': 5, 'buena': 14, 'good': 19, 'it': 7, 'la': 4, 'me': 0, 'Give': 6, 'comer': 2, 'is': 16, 'que': 11, 'si': 24, 'sea': 12, 'lost': 21, 'on': 25, 'at': 22, 'Yo': 23, 'gusta': 1, 'a': 18, 'to': 8, 'idea': 15, 'not': 17, 'get': 20, 'creo': 10, 'en': 3, 'una': 13, 'No': 9}


In [11]:
class BoWClassifier(nn.Module):
    def __init__(self, num_labels, vocab_size):
        super(BoWClassifier, self).__init__()
        
        self.linear = nn.Linear(vocab_size, num_labels)
        
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec))

In [12]:
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [13]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)
for param in model.parameters():
    print(param)

Parameter containing:

Columns 0 to 9 
 0.0164  0.1504  0.1733  0.1007 -0.0252  0.0756  0.1462 -0.1581  0.0271  0.0357
 0.1335  0.0349 -0.1252 -0.0722  0.0484 -0.1848  0.1609  0.0333 -0.1937  0.1243

Columns 10 to 19 
 0.1949  0.0740  0.0664  0.0738  0.1539 -0.0534 -0.0310 -0.0658 -0.0635  0.0008
 0.1490  0.0143  0.0681 -0.0264 -0.0267  0.0253 -0.1395  0.0630  0.1234 -0.1875

Columns 20 to 25 
 0.1679 -0.0210  0.0826 -0.1886 -0.1718 -0.0398
-0.1286  0.0607 -0.0294 -0.0537 -0.1331  0.0809
[torch.FloatTensor of size 2x26]

Parameter containing:
 0.1479
-0.1026
[torch.FloatTensor of size 2]



In [14]:
print(model.parameters)

<bound method Module.parameters of BoWClassifier (
  (linear): Linear (26 -> 2)
)>
